In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error


In [ ]:
def get_xyz_data(filename):
    pos_data = []
    lat_data = []
    with open(filename) as f:
        for line in f.readlines():
            x = line.split()
            if x[0] == 'atom':
                pos_data.append([np.array(x[1:4], dtype=np.float),x[4]])
            elif x[0] == 'lattice_vector':
                lat_data.append(np.array(x[1:4], dtype=np.float))
    
    A = np.transpose(lat_data)
    B = np.linalg.inv(A)
    R = pos_data[0][0]
    return np.matmul(B, R)

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
for c in train.columns:
    if(c.find('angle')!=-1):
        print(c)
        train[c] = np.radians(train[c])
        test[c] = np.radians(test[c])
traindata = np.zeros((train.shape[0],3))
for i,idx in enumerate(train.id.values):
    fn = "../input/train/{}/geometry.xyz".format(idx)
    data = get_xyz_data(fn)
    traindata[i,:] = data
testdata = np.zeros((test.shape[0],3))
for i,idx in enumerate(test.id.values):
    fn = "../input/test/{}/geometry.xyz".format(idx)
    data = get_xyz_data(fn)
    testdata[i,:] = data
train['a0'] = 0
train['a1'] = 0
train['a2'] = 0
train[['a0','a1','a2']]= traindata
test['a0'] = 0
test['a1'] = 0
test['a2'] = 0
test[['a0','a1','a2']] = testdata
train.number_of_total_atoms = np.log(train.number_of_total_atoms)
test.number_of_total_atoms = np.log(test.number_of_total_atoms)
alldata = pd.concat([train,test])


alldata = pd.concat([alldata.drop(['spacegroup'], axis=1), 
                   pd.get_dummies(alldata['spacegroup'], prefix='SG')], axis=1)

train = alldata[:train.shape[0]].copy()
test = alldata[train.shape[0]:].copy()
target_fe = np.log1p(train.formation_energy_ev_natom)
target_be = np.log1p(train.bandgap_energy_ev)
del train['formation_energy_ev_natom'], train['bandgap_energy_ev'], train['id'], test['id']

In [ ]:
def FE1(data):
    v = pd.DataFrame()
    v["i0"] = 0.097734*(((((((data["SG_194"]) + (data["SG_227"]))) + ((((3.0) + (np.sin((((data["percent_atom_in"]) * 2.0)))))/2.0)))) - (data["percent_atom_ga"])))
    v["i1"] = 0.100000*((((np.sin(((((4.0)) * (data["percent_atom_al"]))))) + (np.tanh(((((data["SG_227"]) + (((data["a1"]) + (-2.0))))/2.0)))))/2.0))
    v["i2"] = 0.100000*((((((data["percent_atom_ga"]) * (np.cos(((1.93346190452575684)))))) + (((((((((data["lattice_angle_gamma_degree"]) > (data["lattice_angle_beta_degree"]))*1.)) + (data["a1"]))/2.0)) / 2.0)))/2.0))
    v["i3"] = 0.100000*(((((((data["percent_atom_al"]) * 2.0)) - (((data["percent_atom_in"]) * (data["percent_atom_in"]))))) * (((np.cos((data["lattice_angle_beta_degree"]))) + (data["percent_atom_in"])))))
    v["i4"] = 0.100000*(((data["percent_atom_ga"]) * (((np.sin((np.sin((np.sin((data["SG_206"]))))))) - (((data["percent_atom_al"]) * (((data["percent_atom_al"]) * 2.0))))))))
    v["i5"] = 0.098007*((((np.sin((((data["SG_206"]) * (data["lattice_vector_2_ang"]))))) + ((((((((data["a2"]) < (data["a0"]))*1.)) * (data["a0"]))) / 2.0)))/2.0))
    v["i6"] = 0.088689*(((np.cos(((6.0)))) + (np.cos((((((data["lattice_angle_beta_degree"]) + (np.tanh((data["lattice_angle_beta_degree"]))))) + (data["percent_atom_al"])))))))
    v["i7"] = 0.096386*(((np.tanh((data["percent_atom_in"]))) * ((((np.cos((data["SG_12"]))) < (np.tanh((data["percent_atom_in"]))))*1.))))
    v["i8"] = 0.100000*(((((np.cos((data["lattice_angle_beta_degree"]))) + ((((((data["lattice_vector_1_ang"]) * (data["percent_atom_ga"]))) < (np.cos((data["lattice_vector_1_ang"]))))*1.)))) * (data["percent_atom_ga"])))
    v["i9"] = 0.100000*(((((data["lattice_vector_2_ang"]) * ((((-1.0*((data["a0"])))) + (((data["a0"]) * (np.tanh((3.0))))))))) * (data["a0"])))
    v["i10"] = 0.100000*(((data["a0"]) * (np.sin(((((-1.0*(((6.16540956497192383))))) - (((np.sin((data["lattice_vector_2_ang"]))) * (np.sin((data["lattice_vector_2_ang"])))))))))))
    v["i11"] = 0.100000*(((((((((np.sin(((6.46666765213012695)))) > (((data["percent_atom_in"]) / 2.0)))*1.)) > (data["SG_12"]))*1.)) * (((np.tanh((data["percent_atom_in"]))) / 2.0))))
    v["i12"] = 0.100000*((((np.tanh((-2.0))) + (np.tanh(((((((data["percent_atom_in"]) > (0.0))*1.)) - (np.sin((-2.0))))))))/2.0))
    v["i13"] = 0.100000*(((data["percent_atom_in"]) * (((data["percent_atom_in"]) * (((data["percent_atom_in"]) * (((data["percent_atom_in"]) - (np.sin((np.cos((data["SG_227"])))))))))))))
    v["i14"] = 0.098593*((((((((((data["percent_atom_in"]) > (np.cos((np.sin((data["a1"]))))))*1.)) / 2.0)) * (data["a1"]))) * (np.tanh((data["a1"])))))
    v["i15"] = 0.100000*((((((((1.81121039390563965)) < (((data["percent_atom_ga"]) * (data["lattice_angle_gamma_degree"]))))*1.)) + (((data["percent_atom_in"]) * (((data["percent_atom_ga"]) * (data["SG_206"]))))))/2.0))
    v["i16"] = 0.100000*(((data["a1"]) * ((((((((data["a1"]) * (data["a1"]))) + (np.tanh((np.sin(((4.31349372863769531)))))))/2.0)) * (data["a1"])))))
    v["i17"] = 0.100000*(((((((0.18124227225780487)) > (data["percent_atom_al"]))*1.)) * (data["percent_atom_al"])))
    v["i18"] = 0.094198*(((-1.0) + (np.tanh(((((((((data["percent_atom_al"]) > ((-1.0*((data["a0"])))))*1.)) - (-1.0))) + (data["percent_atom_al"])))))))
    v["i19"] = 0.098965*(np.sin((((((((((data["percent_atom_al"]) > ((-1.0*((data["SG_227"])))))*1.)) > (np.sin((data["lattice_vector_1_ang"]))))*1.)) + (np.tanh((-2.0)))))))
    v["i20"] = 0.085446*(((((np.tanh((data["SG_194"]))) / 2.0)) * ((((((np.tanh((np.tanh((data["SG_194"]))))) / 2.0)) > (np.cos((data["lattice_vector_3_ang"]))))*1.))))
    v["i21"] = 0.100000*(((data["percent_atom_ga"]) * (((data["percent_atom_ga"]) * (np.sin(((((-1.0*((data["percent_atom_al"])))) - ((12.38286399841308594))))))))))
    v["i22"] = 0.100000*(((((((data["a0"]) / 2.0)) * (np.cos((data["a0"]))))) * (((((data["a1"]) / 2.0)) * (np.cos((data["number_of_total_atoms"])))))))
    v["i23"] = 0.100000*(((((-1.0) + (np.sin((data["lattice_angle_beta_degree"]))))) * (np.sin((np.sin((((((data["a0"]) * 2.0)) * 2.0))))))))
    v["i24"] = 0.100000*(((data["SG_227"]) * ((((2.52827239036560059)) * ((((np.tanh(((2.51206827163696289)))) + (np.sin((((-2.0) + (data["percent_atom_al"]))))))/2.0))))))
    v["i25"] = 0.100000*(np.tanh(((((((data["percent_atom_ga"]) < (np.sin(((((((data["percent_atom_al"]) < ((0.00149130856152624)))*1.)) * (data["SG_167"]))))))*1.)) * (data["percent_atom_ga"])))))
    v["i26"] = 0.100000*((((np.tanh((-3.0))) + (np.cos((((((((data["lattice_vector_2_ang"]) < ((4.90194511413574219)))*1.)) + (((data["SG_167"]) - (data["percent_atom_ga"]))))/2.0)))))/2.0))
    v["i27"] = 0.091717*(((np.tanh((((np.tanh((data["percent_atom_al"]))) - (data["percent_atom_ga"]))))) * (((data["SG_206"]) * (((np.tanh((data["percent_atom_al"]))) / 2.0))))))
    v["i28"] = 0.100000*((((((((((-1.0) + (np.cos((np.sin((data["a0"]))))))/2.0)) * (data["percent_atom_in"]))) * (data["percent_atom_in"]))) * (data["percent_atom_in"])))
    v["i29"] = 0.100000*(((((data["number_of_total_atoms"]) * (((2.0) - (data["a0"]))))) * (((1.0) - (np.tanh((data["number_of_total_atoms"])))))))
    v["i30"] = 0.081051*(((((data["lattice_angle_gamma_degree"]) * 2.0)) * (((((((np.tanh((3.0))) - (1.0))) * (data["percent_atom_al"]))) * (data["lattice_angle_gamma_degree"])))))
    v["i31"] = 0.100000*((((((-1.0*((data["lattice_vector_2_ang"])))) + ((5.96246290206909180)))) * (((np.tanh((data["lattice_vector_2_ang"]))) + ((-1.0*((np.tanh((data["number_of_total_atoms"]))))))))))
    v["i32"] = 0.100000*((((np.tanh((-3.0))) + (np.cos((((data["a0"]) * (((np.sin(((6.81815528869628906)))) / 2.0)))))))/2.0))
    v["i33"] = 0.100000*(((2.0) + (((np.tanh((((((((data["percent_atom_ga"]) - (data["a0"]))) + (-3.0))) - (data["a0"]))))) * 2.0))))
    v["i34"] = 0.091424*((((((((8.19109630584716797)) / 2.0)) * (data["lattice_vector_1_ang"]))) * (np.sin((((data["SG_167"]) * (((data["lattice_angle_beta_degree"]) * ((10.0))))))))))
    v["i35"] = 0.073608*(((((np.cos((data["lattice_angle_gamma_degree"]))) + (((1.0) - (data["SG_33"]))))) * (((1.0) + (np.cos((3.0)))))))
    v["i36"] = 0.100000*(((((((((data["percent_atom_in"]) * (((np.sin((data["lattice_vector_3_ang"]))) * 2.0)))) * (data["SG_194"]))) * (data["percent_atom_in"]))) * (data["percent_atom_in"])))
    v["i37"] = 0.095214*(((((1.0) + (np.cos((-3.0))))) * (((((((data["SG_194"]) - (data["a1"]))) * 2.0)) + (data["SG_194"])))))
    v["i38"] = 0.100000*(((((((data["SG_227"]) * (((((data["a0"]) * (np.cos((data["lattice_angle_alpha_degree"]))))) * 2.0)))) - (np.cos((data["lattice_angle_alpha_degree"]))))) * 2.0))
    v["i39"] = 0.100000*(((((data["percent_atom_al"]) - ((((data["percent_atom_al"]) > (((data["SG_227"]) / 2.0)))*1.)))) * (((-1.0) + (np.tanh((2.0)))))))
    v["i40"] = 0.100000*((((((0.00501752039417624)) * (-3.0))) * (((data["lattice_angle_gamma_degree"]) - ((((data["SG_227"]) > ((-1.0*((data["percent_atom_al"])))))*1.))))))
    v["i41"] = 0.100000*((((((0.00115513824857771)) * ((0.00115513824857771)))) - (((((data["lattice_vector_3_ang"]) * ((0.00115513824857771)))) * (data["SG_12"])))))
    v["i42"] = 0.069799*(((1.0) - (np.tanh(((((((((data["lattice_angle_gamma_degree"]) * 2.0)) * 2.0)) + ((((data["lattice_angle_gamma_degree"]) + (data["number_of_total_atoms"]))/2.0)))/2.0))))))
    v["i43"] = 0.095995*(((((((7.83266353607177734)) > (data["lattice_vector_1_ang"]))*1.)) * (((((np.cos((data["lattice_angle_alpha_degree"]))) * ((9.87982559204101562)))) * ((7.83266353607177734))))))
    v["i44"] = 0.088670*(((1.0) + (np.tanh((((((((-3.0) + (((((np.sin((data["lattice_vector_1_ang"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0))))))
    v["i45"] = 0.071498*((((((((data["SG_194"]) + (1.0))/2.0)) + (data["SG_194"]))) * (((np.sin((data["lattice_angle_beta_degree"]))) + (np.tanh((-3.0)))))))
    v["i46"] = 0.100000*(((((((data["a0"]) - (np.cos((((data["a0"]) * (data["lattice_angle_alpha_degree"]))))))) * (np.cos((data["lattice_angle_alpha_degree"]))))) * (data["a0"])))
    v["i47"] = 0.060227*(((data["a1"]) * ((((14.15450668334960938)) * (((np.cos((data["lattice_angle_alpha_degree"]))) * (np.cos((data["lattice_angle_alpha_degree"])))))))))
    v["i48"] = 0.070268*(((((np.tanh(((((3.69988656044006348)) + (-1.0))))) + (-1.0))) * (((np.tanh((data["SG_33"]))) + (-1.0)))))
    v["i49"] = 0.100000*(((((-3.0) * ((0.00034689911990426)))) * (((data["SG_33"]) + ((((((data["percent_atom_in"]) < ((0.00034332284121774)))*1.)) * ((14.99365615844726562))))))))
    v["i50"] = 0.100000*(((((((data["percent_atom_al"]) * (data["percent_atom_al"]))) * (((np.sin((data["percent_atom_ga"]))) * (data["percent_atom_ga"]))))) * (data["percent_atom_al"])))
    v["i51"] = 0.074390*(((((np.tanh((2.0))) + (-1.0))) * ((((np.sin(((2.81946730613708496)))) < (((np.sin((data["lattice_vector_2_ang"]))) * 2.0)))*1.))))
    v["i52"] = 0.088787*(((((np.cos((3.0))) - (-1.0))) * (((((np.cos((data["SG_33"]))) - (-1.0))) * (data["a1"])))))
    v["i53"] = 0.100000*(((((np.sin((np.sin(((((6.0)) * ((((7.0)) * (data["percent_atom_ga"]))))))))) * (data["lattice_vector_3_ang"]))) * ((0.00216007279232144))))
    v["i54"] = 0.100000*(((-1.0) + (np.tanh((((np.sin(((((((6.11344575881958008)) * (data["percent_atom_ga"]))) - (data["SG_194"]))))) - (-3.0)))))))
    v["i55"] = 0.095273*((((((((np.cos(((7.77632999420166016)))) < (data["percent_atom_ga"]))*1.)) + (np.cos(((3.39663577079772949)))))) * (np.tanh((np.tanh((data["percent_atom_ga"])))))))
    v["i56"] = 0.100000*(((np.cos(((11.00160503387451172)))) * (np.cos((((((data["lattice_vector_3_ang"]) + (((data["lattice_vector_3_ang"]) + (data["lattice_vector_3_ang"]))))) * (data["lattice_vector_3_ang"])))))))
    v["i57"] = 0.100000*(((((((np.tanh(((((data["lattice_vector_3_ang"]) + (-3.0))/2.0)))) - (1.0))) * (data["percent_atom_ga"]))) * (data["a0"])))
    v["i58"] = 0.100000*((((((np.tanh((-2.0))) + (1.0))/2.0)) * ((((data["SG_12"]) + ((((data["SG_12"]) + (data["SG_194"]))/2.0)))/2.0))))
    v["i59"] = 0.029517*((((((((data["SG_194"]) * (-3.0))) + (((data["SG_167"]) * (-3.0))))/2.0)) * ((0.00892639346420765))))
    v["i60"] = 0.040535*(((-2.0) * ((((3.0) + ((-1.0*((((((7.0)) + (np.cos((3.0))))/2.0))))))/2.0))))
    v["i61"] = 0.100000*((((((np.tanh((data["lattice_vector_1_ang"]))) > (np.cos((((data["lattice_vector_3_ang"]) + (data["SG_194"]))))))*1.)) - (np.tanh((data["lattice_vector_1_ang"])))))
    v["i62"] = 0.079918*((((1.0) + (np.tanh((((((data["lattice_vector_3_ang"]) / 2.0)) - (data["lattice_vector_3_ang"]))))))/2.0))
    v["i63"] = 0.030670*(((np.sin((data["lattice_vector_3_ang"]))) * ((((-2.0) + (((np.tanh((((np.tanh((data["lattice_vector_1_ang"]))) * 2.0)))) * 2.0)))/2.0))))
    v["i64"] = 0.071830*(((((data["percent_atom_al"]) * 2.0)) * (((data["lattice_vector_1_ang"]) * (((((1.0) - (np.tanh((data["number_of_total_atoms"]))))) * (data["percent_atom_al"])))))))
    v["i65"] = 0.100000*(((((((np.sin(((3.14143610000610352)))) * (data["lattice_vector_1_ang"]))) * (np.sin(((((14.60924148559570312)) * (data["lattice_vector_1_ang"]))))))) * (data["lattice_vector_1_ang"])))
    v["i66"] = 0.100000*(((data["SG_167"]) * (((np.tanh(((((data["lattice_vector_1_ang"]) + (np.sin((((data["lattice_vector_1_ang"]) * ((8.84530162811279297)))))))/2.0)))) + (-1.0)))))
    v["i67"] = 0.100000*(((np.tanh((data["lattice_vector_2_ang"]))) - (np.tanh((((((((4.0)) < ((((7.62273597717285156)) * (data["a0"]))))*1.)) + (3.0)))))))
    v["i68"] = 0.091912*(((((np.tanh((np.tanh((((np.sin((data["lattice_vector_2_ang"]))) * 2.0)))))) * (np.cos((data["lattice_angle_beta_degree"]))))) * (np.cos((data["SG_227"])))))
    v["i69"] = 0.080621*(((((((((-1.0) + (np.sin((data["lattice_angle_alpha_degree"]))))) * 2.0)) * (((-2.0) + (((data["a2"]) * 2.0)))))) * 2.0))
    return v.sum(axis=1).values

def FE2(data):
    v = pd.DataFrame()
    v["i0"] = 0.100000*(((((data["SG_227"]) - (data["percent_atom_ga"]))) + (((data["SG_194"]) + (np.sqrt((np.log((17.0)))))))))
    v["i1"] = 0.093768*(np.minimum( ((((data["percent_atom_ga"]) - ((((((((0.602564) / 2.0)) * (((0.602564) / 2.0)))) > (data["percent_atom_in"]))*1.))))),  ((0.160714))))
    v["i2"] = 0.094139*(np.minimum( ((((((((data["percent_atom_al"]) * 2.0)) * 2.0)) + (((data["a1"]) - (data["percent_atom_in"])))))),  ((data["percent_atom_in"]))))
    v["i3"] = 0.100000*(np.cos((np.minimum( ((data["lattice_angle_beta_degree"])),  ((((2.0) + (np.sin((((data["lattice_angle_beta_degree"]) * (data["lattice_angle_beta_degree"]))))))))))))
    v["i4"] = 0.100000*(((((data["percent_atom_in"]) * (data["percent_atom_in"]))) * (((data["number_of_total_atoms"]) - (((((data["number_of_total_atoms"]) / 2.0)) * (((data["number_of_total_atoms"]) / 2.0))))))))
    v["i5"] = 0.094511*(((((data["SG_206"]) - (((((data["percent_atom_ga"]) * (data["percent_atom_ga"]))) / 2.0)))) * (((data["percent_atom_ga"]) / 2.0))))
    v["i6"] = 0.082516*(((((np.log((data["lattice_angle_gamma_degree"]))) * (data["SG_227"]))) * (((data["lattice_angle_gamma_degree"]) * (0.160714)))))
    v["i7"] = 0.097753*((-1.0*((((data["a0"]) * ((((((0.012658) > (data["a2"]))*1.)) + (((0.012658) * 2.0)))))))))
    v["i8"] = 0.093788*(np.minimum( ((((data["a2"]) * (data["a2"])))),  ((((np.sin((3.0))) - (((((data["a2"]) / 2.0)) * (((data["a2"]) / 2.0)))))))))
    v["i9"] = 0.094823*(((((data["lattice_angle_gamma_degree"]) * (data["lattice_angle_gamma_degree"]))) * ((((((data["a1"]) < (np.cos((data["lattice_angle_gamma_degree"]))))*1.)) * (0.205882)))))
    v["i10"] = 0.078590*(np.log((np.tanh((((((((data["percent_atom_al"]) + (data["SG_227"]))) * (31.006277))) + ((1.14036822319030762))))))))
    v["i11"] = 0.100000*(((np.minimum( ((data["a1"])),  ((np.minimum( ((0.228070)),  (((((0.602564) + (np.sin((data["lattice_vector_1_ang"]))))/2.0)))))))) * (np.minimum( ((data["a1"])),  ((np.minimum( ((0.228070)),  (((((0.602564) + (np.sin((data["lattice_vector_1_ang"]))))/2.0))))))))))
    v["i12"] = 0.100000*(np.minimum( (((-1.0*((((((((0.012658) - (data["SG_227"]))) - (data["SG_33"]))) * 2.0)))))),  ((0.012658))))
    v["i13"] = 0.080738*(((((0.160714) * ((((1.570796) < (data["lattice_angle_alpha_degree"]))*1.)))) * (np.cos((((data["a2"]) * 2.0))))))
    v["i14"] = 0.082164*((-1.0*((((((((np.tanh((np.tanh((data["percent_atom_al"]))))) - (np.sin((data["SG_194"]))))) * (((np.tanh((np.tanh((data["percent_atom_al"]))))) - (np.sin((data["SG_194"]))))))) * (((((np.tanh((np.tanh((data["percent_atom_al"]))))) - (np.sin((data["SG_194"]))))) * (((np.tanh((np.tanh((data["percent_atom_al"]))))) - (np.sin((data["SG_194"]))))))))))))
    v["i15"] = 0.100000*((((((((data["a0"]) < (0.434211))*1.)) * (np.exp((((data["SG_227"]) - (4.153850))))))) * 2.0))
    v["i16"] = 0.100000*(np.minimum( (((((((((data["a2"]) > (data["percent_atom_al"]))*1.)) - (0.012658))) - (0.012658)))),  ((0.012658))))
    v["i17"] = 0.097109*(((((0.012658) - ((((((0.012658) + (data["lattice_vector_2_ang"]))) > ((10.04879188537597656)))*1.)))) * ((0.30205255746841431))))
    v["i18"] = 0.100000*(((data["percent_atom_in"]) * ((((((data["percent_atom_in"]) > ((((data["percent_atom_al"]) < (0.500000))*1.)))*1.)) * (data["percent_atom_in"])))))
    v["i19"] = 0.100000*(((np.minimum( ((0.012658)),  ((np.tanh((np.tanh((((((data["SG_206"]) - (0.012658))) / 2.0))))))))) * 2.0))
    v["i20"] = 0.100000*(((data["SG_206"]) * (((np.tanh((np.sqrt((data["percent_atom_al"]))))) * (((data["percent_atom_al"]) - (0.500000)))))))
    v["i21"] = 0.092440*(((((((data["percent_atom_in"]) * (data["percent_atom_in"]))) - (data["percent_atom_al"]))) * (((data["percent_atom_ga"]) * (np.sqrt((data["percent_atom_in"])))))))
    v["i22"] = 0.100000*((-1.0*((((np.minimum( ((((0.318182) * (data["SG_33"])))),  ((((((data["percent_atom_al"]) * (data["percent_atom_al"]))) * (((data["percent_atom_al"]) * (data["percent_atom_al"])))))))) * (np.minimum( ((((0.318182) * (data["SG_33"])))),  ((((((data["percent_atom_al"]) * (data["percent_atom_al"]))) * (((data["percent_atom_al"]) * (data["percent_atom_al"])))))))))))))
    v["i23"] = 0.100000*(((((np.minimum( (((((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0)) * ((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0))))),  ((data["percent_atom_ga"])))) * (np.minimum( (((((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0)) * ((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0))))),  ((data["percent_atom_ga"])))))) * (((np.minimum( (((((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0)) * ((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0))))),  ((data["percent_atom_ga"])))) * (np.minimum( (((((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0)) * ((((data["percent_atom_ga"]) + (np.log((data["lattice_angle_gamma_degree"]))))/2.0))))),  ((data["percent_atom_ga"]))))))))
    v["i24"] = 0.100000*(((0.012658) * ((((((-1.0*(((((data["percent_atom_in"]) < (0.012658))*1.))))) * 2.0)) * 2.0))))
    v["i25"] = 0.100000*(((((0.344086) + (data["SG_206"]))) * (((((((((data["a0"]) * (data["a0"]))) * (((data["a0"]) * (data["a0"]))))) * (((((data["a0"]) * (data["a0"]))) * (((data["a0"]) * (data["a0"]))))))) * (((((((data["a0"]) * (data["a0"]))) * (((data["a0"]) * (data["a0"]))))) * (((((data["a0"]) * (data["a0"]))) * (((data["a0"]) * (data["a0"])))))))))))
    v["i26"] = 0.063489*(((np.sin((((data["lattice_vector_2_ang"]) * (data["lattice_vector_2_ang"]))))) * (((1.570796) - (data["lattice_angle_alpha_degree"])))))
    v["i27"] = 0.100000*(((((1.570796) * (data["percent_atom_al"]))) * ((((((data["percent_atom_al"]) * (data["percent_atom_al"]))) < ((0.01294255536049604)))*1.))))
    v["i28"] = 0.100000*(np.log((np.tanh((((((np.exp((np.minimum( ((data["a2"])),  ((np.sqrt((data["percent_atom_al"])))))))) * (np.exp((np.minimum( ((data["a2"])),  ((np.sqrt((data["percent_atom_al"])))))))))) * 2.0))))))
    v["i29"] = 0.100000*(np.minimum( ((((0.012658) * (((np.sin((data["SG_12"]))) - (np.sin((data["number_of_total_atoms"])))))))),  ((data["percent_atom_al"]))))
    v["i30"] = 0.100000*((-1.0*((np.minimum( (((((((0.012658) < (((data["percent_atom_in"]) * (data["percent_atom_in"]))))*1.)) - (data["percent_atom_in"])))),  ((0.012658)))))))
    v["i31"] = 0.100000*((((((data["SG_206"]) + ((((data["SG_206"]) + (((data["percent_atom_in"]) / 2.0)))/2.0)))/2.0)) * (np.cos((data["lattice_angle_gamma_degree"])))))
    v["i32"] = 0.100000*(np.log((np.tanh((((0.434211) * (((data["lattice_vector_3_ang"]) + (((np.sin((data["lattice_vector_3_ang"]))) * 2.0))))))))))
    v["i33"] = 0.100000*(((((np.cos((np.exp((data["a0"]))))) * (0.012658))) * (np.exp((data["SG_227"])))))
    v["i34"] = 0.075523*(np.minimum( ((0.012658)),  (((((((((data["percent_atom_in"]) - (0.012658))) > (0.160714))*1.)) - (data["percent_atom_in"]))))))
    v["i35"] = 0.099355*(((((np.tanh((((data["percent_atom_al"]) * (data["percent_atom_al"]))))) * (np.cos((((data["lattice_vector_2_ang"]) * 2.0)))))) * (0.160714)))
    v["i36"] = 0.100000*(((((np.minimum( ((0.205882)),  ((data["SG_167"])))) * (data["lattice_vector_2_ang"]))) - (data["SG_167"])))
    v["i37"] = 0.100000*(np.minimum( ((data["percent_atom_in"])),  ((((np.tanh((data["SG_167"]))) * (((data["percent_atom_ga"]) - (((data["percent_atom_in"]) / 2.0)))))))))
    v["i38"] = 0.100000*(((0.012658) * ((((((data["percent_atom_ga"]) < (0.160714))*1.)) - ((((data["SG_33"]) < (data["percent_atom_ga"]))*1.))))))
    v["i39"] = 0.099375*((((((((data["percent_atom_in"]) < (((0.012658) * ((7.0)))))*1.)) * (data["percent_atom_in"]))) * (4.285710)))
    v["i40"] = 0.100000*((-1.0*((((0.012658) * (np.cos((((np.cos((np.sqrt((2.571430))))) / 2.0)))))))))
    v["i41"] = 0.099590*((-1.0*((((((np.sin((np.sin((np.sin((((((data["percent_atom_al"]) * (data["percent_atom_al"]))) * (((data["percent_atom_al"]) * (data["percent_atom_al"]))))))))))) / 2.0)) * (((np.sin((np.sin((np.sin((((((data["percent_atom_al"]) * (data["percent_atom_al"]))) * (((data["percent_atom_al"]) * (data["percent_atom_al"]))))))))))) / 2.0)))))))
    v["i42"] = 0.100000*(((np.minimum( ((np.minimum( ((0.012658)),  (((((data["SG_12"]) > (data["percent_atom_al"]))*1.)))))),  ((data["percent_atom_al"])))) * (data["lattice_angle_beta_degree"])))
    v["i43"] = 0.097929*(((((data["SG_12"]) * (((data["percent_atom_ga"]) - (0.318182))))) * (((data["percent_atom_ga"]) - (data["SG_12"])))))
    v["i44"] = 0.091209*(((((0.344086) * (np.minimum( (((((0.344086) < (data["percent_atom_ga"]))*1.))),  ((((data["SG_206"]) / 2.0))))))) / 2.0))
    v["i45"] = 0.100000*(np.log((np.tanh((((data["lattice_vector_3_ang"]) + (((-3.0) - ((((data["lattice_vector_2_ang"]) > (data["lattice_vector_3_ang"]))*1.))))))))))
    v["i46"] = 0.100000*((((((data["SG_206"]) + (((data["percent_atom_ga"]) * ((-1.0*((data["SG_206"])))))))/2.0)) * (np.log((0.952381)))))
    v["i47"] = 0.100000*(((np.minimum( ((np.tanh((data["percent_atom_al"])))),  (((((np.sin((data["lattice_angle_beta_degree"]))) < (np.cos((data["percent_atom_al"]))))*1.))))) / 2.0))
    v["i48"] = 0.100000*(np.log((np.tanh((((3.0) - (np.minimum( ((3.0)),  (((((data["lattice_vector_3_ang"]) < ((6.07777643203735352)))*1.)))))))))))
    v["i49"] = 0.087361*(((((0.012658) + (((data["percent_atom_in"]) * 2.0)))) * ((((data["percent_atom_in"]) < (((0.160714) / 2.0)))*1.))))
    v["i50"] = 0.100000*(((((((-2.0) * 2.0)) / 2.0)) * (np.minimum( ((data["percent_atom_al"])),  ((np.minimum( ((0.012658)),  ((data["SG_167"])))))))))
    v["i51"] = 0.093749*(((((0.205882) * (0.205882))) * (np.cos((((data["lattice_vector_2_ang"]) + (((((data["lattice_vector_2_ang"]) * (data["lattice_vector_2_ang"]))) / 2.0))))))))
    v["i52"] = 0.090604*(((np.minimum( ((0.160714)),  ((data["SG_12"])))) * (np.sin((np.sin((np.sin((data["lattice_vector_3_ang"])))))))))
    v["i53"] = 0.100000*((((((((data["lattice_angle_gamma_degree"]) < (np.cos((np.exp((data["lattice_vector_1_ang"]))))))*1.)) * (0.160714))) * (0.500000)))
    v["i54"] = 0.100000*(((((((((np.sin((data["SG_167"]))) * (data["lattice_vector_1_ang"]))) * (0.012658))) * (((((np.sin((data["SG_167"]))) * (data["lattice_vector_1_ang"]))) * (0.012658))))) * (data["lattice_vector_1_ang"])))
    v["i55"] = 0.100000*(np.log((np.tanh((((((((((10.20271492004394531)) < (data["lattice_vector_1_ang"]))*1.)) * ((10.20271492004394531)))) + (2.571430)))))))
    v["i56"] = 0.049463*(((0.012658) * (((((((((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))) * (((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))))) * (((((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))) * (((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))))))) * (((((((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))) * (((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))))) * (((((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"]))))) * (((data["lattice_angle_gamma_degree"]) * (np.tanh((data["a0"])))))))))))))
    v["i57"] = 0.068744*(((np.minimum( ((((((data["a2"]) - (data["SG_227"]))) - (data["SG_227"])))),  ((data["a2"])))) * (0.012658)))
    v["i58"] = 0.100000*(((0.012658) * (np.cos((((np.exp((((np.cos((0.012658))) + (data["lattice_vector_3_ang"]))))) * 2.0))))))
    v["i59"] = 0.085818*(((np.log((np.tanh((2.571430))))) * (np.log((((np.tanh((data["lattice_angle_gamma_degree"]))) * (data["lattice_angle_gamma_degree"])))))))
    v["i60"] = 0.100000*(((np.sin((((np.sin((((((data["lattice_vector_3_ang"]) * (data["lattice_vector_3_ang"]))) + (data["a0"]))))) * 2.0)))) * (0.012658)))
    v["i61"] = 0.100000*(((np.sin((((np.exp((data["lattice_vector_3_ang"]))) / 2.0)))) * (((((0.160714) / 2.0)) * (data["SG_194"])))))
    v["i62"] = 0.100000*(((((0.012658) * 2.0)) * (np.cos((((data["lattice_vector_2_ang"]) * (((((data["lattice_vector_2_ang"]) * (data["lattice_vector_2_ang"]))) * (((data["lattice_vector_2_ang"]) * (data["lattice_vector_2_ang"])))))))))))
    v["i63"] = 0.075230*(((((((data["SG_167"]) * (data["a0"]))) * (((data["SG_167"]) * (data["a0"]))))) + (((0.012658) * (0.500000)))))
    v["i64"] = 0.100000*(np.log((np.tanh((((((1.570796) * (1.570796))) + ((((((1.570796) < (data["lattice_angle_alpha_degree"]))*1.)) * 2.0))))))))
    v["i65"] = 0.100000*(((((data["a2"]) * ((4.0)))) * (((0.012658) * ((((data["a0"]) < (0.012658))*1.))))))
    v["i66"] = 0.093417*(((((data["SG_12"]) + (((0.012658) - (data["SG_227"]))))) * (0.012658)))
    v["i67"] = 0.000000*(((data["SG_194"]) * (((-1.0) * (0.012658)))))
    v["i68"] = 0.099922*(((((((((((-1.0) / 2.0)) + (data["a0"]))) * (np.cos((data["lattice_angle_alpha_degree"]))))) * 2.0)) * 2.0))
    v["i69"] = 0.100000*(((((data["lattice_vector_2_ang"]) / 2.0)) * (((0.012658) * ((((data["a0"]) > (np.tanh(((1.07709670066833496)))))*1.))))))
    v["i70"] = 0.045927*(np.log((np.tanh((((((((2.487180) + (data["SG_194"]))) + (data["a1"]))) + (data["SG_194"])))))))
    v["i71"] = 0.052725*(((((np.sqrt((((np.cos((1.570796))) * 2.0)))) * 2.0)) * 2.0))
    v["i72"] = 0.100000*((((((data["lattice_angle_gamma_degree"]) < (np.sin((np.tanh((0.636620))))))*1.)) * (((2.487180) * (0.012658)))))
    v["i73"] = 0.099101*(np.tanh((np.tanh((((((((0.46847116947174072)) > (np.cos((((data["lattice_angle_alpha_degree"]) * (4.250000))))))*1.)) / 2.0))))))
    v["i74"] = 0.078609*(np.minimum( ((np.minimum( ((data["a2"])),  ((np.tanh((np.cos((data["lattice_angle_alpha_degree"]))))))))),  (((((0.952381) > (data["a2"]))*1.)))))
    v["i75"] = 0.100000*((((((((((0.012658) > (data["percent_atom_in"]))*1.)) * (((0.012658) * (data["percent_atom_ga"]))))) * 2.0)) * 2.0))
    v["i76"] = 0.064427*(((3.0) * (np.log((np.tanh((data["number_of_total_atoms"])))))))
    v["i77"] = 0.000000*(np.minimum( ((np.cos((((data["SG_194"]) + (0.602564)))))),  ((0.0))))
    v["i78"] = 0.068959*(np.log((np.tanh(((((((0.160714) < (((data["percent_atom_in"]) + (data["SG_194"]))))*1.)) + (2.0)))))))
    v["i79"] = 0.100000*(np.minimum( ((((np.minimum( ((data["percent_atom_al"])),  (((((data["percent_atom_in"]) < (0.500000))*1.))))) - (0.012658)))),  ((0.012658))))
    v["i80"] = 0.100000*(((data["percent_atom_in"]) * (((((0.228070) * (0.228070))) * (np.cos((((data["percent_atom_al"]) * ((10.0))))))))))
    v["i81"] = 0.100000*(np.minimum( ((0.0)),  ((((((((data["percent_atom_al"]) + (data["SG_206"]))) - (0.012658))) - (0.012658))))))
    v["i82"] = 0.092499*(((data["SG_227"]) * (((((data["a1"]) * (((data["percent_atom_al"]) - (0.344086))))) * (data["a1"])))))
    v["i83"] = 0.100000*(((data["percent_atom_in"]) * (((((((data["SG_206"]) - (data["percent_atom_in"]))) * (((data["SG_206"]) - (data["percent_atom_in"]))))) * (((0.318182) * (0.318182)))))))
    v["i84"] = 0.100000*((-1.0*((((data["percent_atom_in"]) * (np.minimum( ((data["lattice_angle_gamma_degree"])),  (((((0.318310) > (np.sqrt((data["percent_atom_in"]))))*1.))))))))))
    v["i85"] = 0.100000*(np.log((np.tanh((np.maximum( (((2.40907120704650879))),  ((((((data["percent_atom_ga"]) + (data["SG_33"]))) * ((10.0)))))))))))
    v["i86"] = 0.099863*(np.sqrt((((((data["SG_167"]) * 2.0)) * (np.minimum( (((((0.012658) > (data["percent_atom_al"]))*1.))),  ((0.012658))))))))
    v["i87"] = 0.075952*((((((-1.0*((0.012658)))) * (((data["lattice_angle_gamma_degree"]) + (((data["SG_167"]) * 2.0)))))) * (data["percent_atom_in"])))
    v["i88"] = 0.100000*(((((np.minimum( ((((data["percent_atom_al"]) - (0.012658)))),  ((0.0)))) * ((6.0)))) * (data["SG_194"])))
    v["i89"] = 0.100000*(((((((data["SG_194"]) * (np.minimum( ((((data["percent_atom_al"]) - (0.012658)))),  ((0.012658)))))) * 2.0)) * 2.0))
    v["i90"] = 0.090447*(np.minimum( ((data["a0"])),  ((((((np.sin((((data["lattice_vector_1_ang"]) * (data["lattice_vector_1_ang"]))))) * 2.0)) * (0.012658))))))
    v["i91"] = 0.100000*(((((((((np.cos((data["lattice_angle_gamma_degree"]))) * (np.cos((data["lattice_angle_gamma_degree"]))))) * (((np.cos((data["lattice_angle_gamma_degree"]))) * (np.cos((data["lattice_angle_gamma_degree"]))))))) * (((((np.cos((data["lattice_angle_gamma_degree"]))) * (np.cos((data["lattice_angle_gamma_degree"]))))) * (((np.cos((data["lattice_angle_gamma_degree"]))) * (np.cos((data["lattice_angle_gamma_degree"]))))))))) * (((0.012658) * 2.0))))
    v["i92"] = 0.090916*((((((((0.739130) < (((((np.cos((data["lattice_vector_3_ang"]))) * (np.cos((data["lattice_vector_3_ang"]))))) * (((np.cos((data["lattice_vector_3_ang"]))) * (np.cos((data["lattice_vector_3_ang"]))))))))*1.)) * 2.0)) * (0.012658)))
    v["i93"] = 0.083727*(np.log((np.maximum( ((np.tanh((2.571430)))),  ((data["SG_167"]))))))
    v["i94"] = 0.100000*((((((data["percent_atom_al"]) > (data["lattice_angle_gamma_degree"]))*1.)) * (((((4.285710) * (data["lattice_angle_gamma_degree"]))) * (0.012658)))))
    v["i95"] = 0.100000*(((((((0.952381) - (((data["percent_atom_ga"]) * 2.0)))) * (0.012658))) * (((data["SG_167"]) * 2.0))))
    v["i96"] = 0.100000*(np.minimum( ((0.0)),  ((0.228070))))
    v["i97"] = 0.011467*(((-1.0) * (0.012658)))
    v["i98"] = 0.100000*(((0.318310) * (((data["a1"]) * (0.012658)))))
    v["i99"] = 0.027818*((-1.0*((((((0.344086) * (0.012658))) * (np.cos((data["SG_12"]))))))))
    return v.sum(axis=1).values





In [ ]:
def BE1(data):
    v = pd.DataFrame()  
    v["i0"] = 0.098515*((((((11.97265815734863281)) - (((data["percent_atom_in"]) * ((8.01905632019042969)))))) + ((((3.50738978385925293)) * (data["percent_atom_al"])))))
    v["i1"] = 0.100000*(((((((((np.sin((data["SG_167"]))) - (data["SG_227"]))) * 2.0)) + (data["SG_206"]))) - (data["SG_194"])))
    v["i2"] = 0.100000*((((((data["SG_33"]) + (((((1.0) - (data["SG_227"]))) - (data["percent_atom_ga"]))))/2.0)) - (data["percent_atom_ga"])))
    v["i3"] = 0.100000*(((data["percent_atom_al"]) * ((((1.08919882774353027)) * (((np.cos((((np.cos((data["percent_atom_al"]))) * 2.0)))) * 2.0))))))
    v["i4"] = 0.100000*(np.sin((np.sin((np.sin((((((((-3.0) * (np.cos((data["lattice_angle_beta_degree"]))))) * 2.0)) * 2.0))))))))
    v["i5"] = 0.100000*((((((np.cos((((np.sin((data["percent_atom_in"]))) * ((7.80028772354125977)))))) + ((-1.0*((data["SG_227"])))))/2.0)) / 2.0))
    v["i6"] = 0.100000*((((((np.sin((((data["lattice_angle_gamma_degree"]) * 2.0)))) + (((data["SG_167"]) - (((data["a1"]) / 2.0)))))/2.0)) / 2.0))
    v["i7"] = 0.100000*(((np.sin(((0.06088973581790924)))) - (((data["a1"]) * (((np.sin((data["lattice_vector_3_ang"]))) * (data["SG_227"])))))))
    v["i8"] = 0.100000*(((data["percent_atom_in"]) * (((data["percent_atom_in"]) - (np.cos((np.cos((((data["SG_227"]) + (data["lattice_vector_3_ang"])))))))))))
    v["i9"] = 0.098613*(((data["percent_atom_ga"]) * (((((-1.0*((data["percent_atom_in"])))) + (np.sin((((((data["percent_atom_ga"]) * 2.0)) * 2.0)))))/2.0))))
    v["i10"] = 0.100000*(((data["SG_227"]) * (((((data["a2"]) * (((data["a2"]) + (np.cos(((9.57482337951660156)))))))) * 2.0))))
    v["i11"] = 0.100000*((-1.0*(((((((((-2.0) + (((data["number_of_total_atoms"]) / 2.0)))/2.0)) / 2.0)) / 2.0)))))
    v["i12"] = 0.100000*(((((((data["a2"]) * 2.0)) + (-1.0))) * (np.sin((((((data["lattice_angle_alpha_degree"]) * 2.0)) * 2.0))))))
    v["i13"] = 0.095370*(((((data["lattice_angle_gamma_degree"]) * (((((np.cos((data["lattice_angle_beta_degree"]))) * (3.0))) * (data["SG_227"]))))) * 2.0))
    v["i14"] = 0.100000*(((data["a2"]) * (((data["a2"]) * (np.sin((((-3.0) * (np.cos((data["lattice_angle_alpha_degree"])))))))))))
    v["i15"] = 0.093534*(((data["percent_atom_ga"]) * (((data["lattice_angle_gamma_degree"]) * (((np.cos(((9.0)))) + (np.cos((data["a2"])))))))))
    v["i16"] = 0.095566*((((((((np.cos((data["lattice_angle_beta_degree"]))) * (data["a0"]))) + (((data["a1"]) / 2.0)))/2.0)) * (data["a0"])))
    v["i17"] = 0.096738*(np.sin((((data["percent_atom_ga"]) * (((((data["percent_atom_ga"]) * (((data["SG_194"]) * (data["percent_atom_ga"]))))) * 2.0))))))
    v["i18"] = 0.100000*((((np.sin(((10.30771064758300781)))) + (np.cos((np.sin((((np.sin((((data["lattice_vector_3_ang"]) * 2.0)))) * 2.0)))))))/2.0))
    v["i19"] = 0.096171*(((np.sin(((((-1.0*((data["lattice_vector_3_ang"])))) * 2.0)))) * (np.sin(((((6.0)) * (data["SG_194"])))))))
    v["i20"] = 0.100000*(((((((((np.cos((data["lattice_angle_alpha_degree"]))) * (data["a0"]))) * (data["a0"]))) * 2.0)) * (-3.0)))
    v["i21"] = 0.056964*(((((data["lattice_angle_alpha_degree"]) + (((np.sin((((data["a2"]) * 2.0)))) * 2.0)))) * (np.cos((data["lattice_angle_alpha_degree"])))))
    v["i22"] = 0.100000*(((((1.0) - (np.cos((((((data["percent_atom_in"]) * (data["percent_atom_in"]))) * (data["percent_atom_in"]))))))) / 2.0))
    v["i23"] = 0.073960*(((np.cos((data["percent_atom_al"]))) * (((data["SG_167"]) * (((data["percent_atom_al"]) + (np.cos(((8.18572425842285156))))))))))
    v["i24"] = 0.083141*(((data["SG_227"]) * (((data["lattice_angle_gamma_degree"]) * (((data["percent_atom_al"]) * (((data["percent_atom_al"]) - (data["a2"])))))))))
    v["i25"] = 0.100000*((((2.58341145515441895)) * (((data["SG_194"]) * (((((data["percent_atom_ga"]) - (data["percent_atom_al"]))) * (data["percent_atom_in"])))))))
    v["i26"] = 0.100000*(((data["a2"]) * (((data["a2"]) * (((data["a2"]) * (((data["SG_194"]) * 2.0))))))))
    v["i27"] = 0.100000*(((np.sin((np.sin((np.sin((data["percent_atom_ga"]))))))) - (np.sin((np.sin((data["percent_atom_ga"])))))))
    v["i28"] = 0.100000*(((data["a0"]) * (((data["a2"]) * (((data["a0"]) * (((data["a2"]) - (data["a0"])))))))))
    v["i29"] = 0.098281*(((np.sin((np.sin((((data["lattice_angle_beta_degree"]) * 2.0)))))) * (np.sin((((data["lattice_angle_beta_degree"]) * (data["lattice_angle_beta_degree"])))))))
    v["i30"] = 0.100000*(((data["lattice_angle_alpha_degree"]) * (((((data["lattice_angle_alpha_degree"]) * ((-1.0*((np.cos((data["lattice_angle_alpha_degree"])))))))) * (data["a1"])))))
    v["i31"] = 0.096484*(((np.cos((((data["lattice_vector_1_ang"]) * (data["lattice_vector_1_ang"]))))) * ((((4.75409507751464844)) * (np.cos((data["lattice_angle_alpha_degree"])))))))
    v["i32"] = 0.100000*(((((((((np.sin(((((((12.74442577362060547)) * (data["lattice_vector_1_ang"]))) / 2.0)))) / 2.0)) / 2.0)) / 2.0)) / 2.0))
    v["i33"] = 0.100000*(((np.cos(((1.55867493152618408)))) * (((((data["a2"]) * (((data["a2"]) * 2.0)))) - (2.0)))))
    v["i34"] = 0.100000*(((((((np.cos(((7.0)))) - (data["percent_atom_al"]))) * (data["percent_atom_al"]))) * (np.sin((data["percent_atom_al"])))))
    v["i35"] = 0.097363*((((0.00891566462814808)) * (((-3.0) - (np.cos((((data["a2"]) + (((data["a2"]) * 2.0))))))))))
    v["i36"] = 0.100000*(((np.sin(((((10.83255100250244141)) * (((data["lattice_vector_1_ang"]) * (data["lattice_vector_1_ang"]))))))) * (np.cos((data["lattice_angle_alpha_degree"])))))
    v["i37"] = 0.100000*(((np.cos((data["lattice_angle_alpha_degree"]))) + (((data["a0"]) * (np.sin((((((data["lattice_angle_alpha_degree"]) * 2.0)) * 2.0))))))))
    v["i38"] = 0.070483*(((np.cos((data["SG_227"]))) * (((np.sin((-3.0))) * (data["SG_227"])))))
    v["i39"] = 0.097128*((((0.02267361246049404)) * (((-3.0) * (np.sin((((data["lattice_vector_1_ang"]) * (data["lattice_vector_1_ang"])))))))))
    v["i40"] = 0.100000*(((np.sin(((6.28206396102905273)))) * ((((-1.0*((np.cos((((data["lattice_vector_1_ang"]) * 2.0))))))) * ((13.75409507751464844))))))
    v["i41"] = 0.100000*(((((np.sin((data["lattice_vector_1_ang"]))) - (np.sin((np.sin((np.sin((data["lattice_vector_1_ang"]))))))))) * (data["a0"])))
    v["i42"] = 0.066439*(((np.sin((data["percent_atom_in"]))) * (((((((-2.0) + (data["percent_atom_in"]))/2.0)) + (np.cos((data["a0"]))))/2.0))))
    v["i43"] = 0.048271*(((np.sin(((((9.0)) * 2.0)))) * (np.sin((((((data["lattice_angle_gamma_degree"]) * 2.0)) * ((9.0))))))))
    v["i44"] = 0.100000*(((((((np.sin((((data["lattice_vector_3_ang"]) * ((7.19170856475830078)))))) * 2.0)) * 2.0)) * (np.cos((data["lattice_angle_alpha_degree"])))))
    v["i45"] = 0.100000*(((np.sin((((((data["percent_atom_in"]) * ((12.12992477416992188)))) * 2.0)))) * (((np.cos((data["lattice_angle_alpha_degree"]))) * 2.0))))
    v["i46"] = 0.100000*(((((np.sin((((np.sin((((data["percent_atom_in"]) * 2.0)))) * 2.0)))) + (-1.0))) * (data["percent_atom_in"])))
    v["i47"] = 0.077769*((((0.02399325929582119)) * ((((((data["lattice_angle_gamma_degree"]) * (((3.0) * (data["a1"]))))) + (data["lattice_angle_gamma_degree"]))/2.0))))
    v["i48"] = 0.100000*((((-1.0*((np.sin((((np.sin(((6.28128099441528320)))) * (data["SG_12"])))))))) * 2.0))
    v["i49"] = 0.100000*(((((((data["a0"]) * (np.cos((data["lattice_angle_alpha_degree"]))))) * (np.cos((((data["lattice_vector_3_ang"]) * 2.0)))))) * 2.0))
    v["i50"] = 0.100000*((-1.0*((((((((data["percent_atom_ga"]) * (((data["SG_227"]) / 2.0)))) * (data["lattice_angle_gamma_degree"]))) * (data["percent_atom_ga"]))))))
    v["i51"] = 0.088357*(((np.sin(((3.0)))) * (np.sin(((((10.33238315582275391)) * ((-1.0*((data["percent_atom_al"]))))))))))
    v["i52"] = 0.100000*(((((data["SG_206"]) * (((data["percent_atom_in"]) - (np.cos((np.sin(((14.78307723999023438)))))))))) * (data["percent_atom_in"])))
    v["i53"] = 0.100000*(((((((data["a1"]) + (data["a1"]))) * ((((0.03250480443239212)) + (data["SG_227"]))))) * ((0.03250122815370560))))
    v["i54"] = 0.100000*(((((data["SG_227"]) * (data["percent_atom_ga"]))) * (np.cos(((((6.0)) * (((data["a2"]) * 2.0))))))))
    v["i55"] = 0.100000*(((((data["SG_227"]) * (data["a1"]))) * (np.sin((np.sin((((data["lattice_vector_3_ang"]) * ((4.78738307952880859))))))))))
    v["i56"] = 0.050693*(((data["lattice_angle_gamma_degree"]) * ((((1.0) + (np.cos((((((-3.0) * 2.0)) / 2.0)))))/2.0))))
    v["i57"] = 0.100000*(((((((-1.0) - (np.cos((3.0))))) * ((3.15338802337646484)))) * (np.cos((data["lattice_angle_gamma_degree"])))))
    v["i58"] = 0.098672*(((np.cos((data["lattice_angle_alpha_degree"]))) * (((np.sin((((data["lattice_vector_3_ang"]) * (((data["lattice_vector_3_ang"]) * 2.0)))))) * 2.0))))
    v["i59"] = 0.098027*(((((np.sin(((11.24273967742919922)))) + (1.0))) * (np.sin((((data["lattice_vector_3_ang"]) * ((11.24273967742919922))))))))
    v["i60"] = 0.092909*((((((((-1.0) + (np.sin((data["lattice_angle_gamma_degree"]))))/2.0)) / 2.0)) / 2.0))
    v["i61"] = 0.050615*(((np.sin(((9.0)))) * (np.cos(((((9.0)) * ((((9.0)) * (data["lattice_angle_alpha_degree"])))))))))
    v["i62"] = 0.100000*((((((-1.0*((np.cos((data["lattice_angle_alpha_degree"])))))) * 2.0)) * ((-1.0*((((np.cos((data["lattice_angle_alpha_degree"]))) * 2.0)))))))
    v["i63"] = 0.054073*(((((((np.sin((data["SG_227"]))) / 2.0)) / 2.0)) / 2.0))
    v["i64"] = 0.099004*((((((-1.0) + (np.cos((((np.cos((-2.0))) + (data["percent_atom_al"]))))))/2.0)) * (-1.0)))
    v["i65"] = 0.100000*((((-1.0*((((((data["percent_atom_in"]) * ((0.06090761721134186)))) / 2.0))))) / 2.0))
    v["i66"] = 0.100000*(np.sin(((((((6.27864885330200195)) / 2.0)) * 2.0))))
    v["i67"] = 0.099980*(((((data["SG_167"]) - (((-1.0) / 2.0)))) * (((-1.0) - (np.cos((3.0)))))))
    v["i68"] = 0.072456*(((np.sin((((data["lattice_vector_3_ang"]) * (((data["lattice_vector_3_ang"]) - ((6.04490995407104492)))))))) * (((data["SG_227"]) / 2.0))))
    v["i69"] = 0.093319*(((((np.cos((data["lattice_angle_alpha_degree"]))) * (((data["a2"]) + (((data["a2"]) - (data["lattice_angle_alpha_degree"]))))))) * 2.0))
    v["i70"] = 0.100000*(((((data["lattice_angle_alpha_degree"]) * 2.0)) * (((((np.sin((data["lattice_angle_alpha_degree"]))) - (1.0))) * (data["a2"])))))
    v["i71"] = 0.100000*(((data["SG_227"]) * (((np.sin(((((8.02241420745849609)) * (data["lattice_vector_3_ang"]))))) * (np.cos(((8.02241706848144531))))))))
    v["i72"] = 0.100000*((((14.46233177185058594)) * ((((14.33283805847167969)) * (((((data["SG_206"]) * 2.0)) * (np.cos((data["lattice_angle_gamma_degree"])))))))))
    v["i73"] = 0.100000*(((((np.sin((((data["lattice_vector_3_ang"]) * ((9.0)))))) * 2.0)) * (np.sin(((9.44896697998046875))))))
    v["i74"] = 0.100000*(((((data["percent_atom_ga"]) + (data["percent_atom_ga"]))) * (((np.cos((data["lattice_angle_beta_degree"]))) * (data["SG_227"])))))
    v["i75"] = 0.100000*((((-1.0*((np.cos((data["lattice_angle_alpha_degree"])))))) * (np.cos((((np.cos((data["percent_atom_in"]))) * (data["lattice_vector_3_ang"])))))))
    v["i76"] = 0.100000*(((np.sin(((((-1.0*((data["lattice_vector_3_ang"])))) * (np.cos((data["lattice_vector_3_ang"]))))))) * (np.cos((data["lattice_angle_alpha_degree"])))))
    v["i77"] = 0.098340*((((((-1.0*((((data["percent_atom_in"]) * 2.0))))) * (np.cos((data["lattice_angle_alpha_degree"]))))) * (data["percent_atom_in"])))
    v["i78"] = 0.100000*(((np.sin((((data["number_of_total_atoms"]) * (3.0))))) * (((data["percent_atom_ga"]) - (np.sin((data["percent_atom_ga"])))))))
    v["i79"] = 0.026724*(((((((-1.0) * (np.sin(((9.42473411560058594)))))) * ((((14.24549484252929688)) * ((9.42473030090332031)))))) * 2.0))
    v["i80"] = 0.100000*(((((np.cos((3.0))) + (1.0))) * (np.cos((((data["lattice_vector_1_ang"]) * ((6.98438072204589844))))))))
    v["i81"] = 0.009299*((((0.11098626255989075)) * ((0.11098268628120422))))
    v["i82"] = 0.091053*(((((data["percent_atom_in"]) * (((1.0) - (((((data["number_of_total_atoms"]) / 2.0)) / 2.0)))))) * (data["percent_atom_in"])))
    v["i83"] = 0.002579*((((((-1.0*((1.0)))) + (data["SG_33"]))) * (((np.cos((3.0))) + (1.0)))))
    v["i84"] = 0.091815*(((np.sin((data["lattice_vector_2_ang"]))) * (np.sin((np.sin((((np.cos((data["lattice_vector_2_ang"]))) * (data["SG_12"])))))))))
    v["i85"] = 0.057746*(((((((np.cos((data["SG_194"]))) - (((data["percent_atom_in"]) * 2.0)))) * 2.0)) * ((((0.01176238339394331)) * 2.0))))
    v["i86"] = 0.086267*(((np.cos((((data["lattice_angle_beta_degree"]) / 2.0)))) * (((np.cos((data["lattice_angle_beta_degree"]))) * (data["SG_227"])))))
    v["i87"] = 0.100000*((-1.0*((((np.cos((data["lattice_angle_alpha_degree"]))) * (np.cos((((data["lattice_angle_alpha_degree"]) * ((8.0)))))))))))
    v["i88"] = 0.049560*(((data["lattice_vector_2_ang"]) * ((-1.0*(((((0.00759601779282093)) * (data["SG_227"]))))))))
    v["i89"] = 0.100000*((((((0.00610470911487937)) * 2.0)) * (((np.cos(((((((8.85007953643798828)) * (data["lattice_vector_3_ang"]))) * 2.0)))) * 2.0))))
    v["i90"] = 0.100000*(((np.cos((data["lattice_angle_alpha_degree"]))) * (np.sin((((((data["lattice_angle_beta_degree"]) * ((14.13362503051757812)))) * (data["lattice_vector_3_ang"])))))))
    return v.sum(axis=1).values

def BE2(data):
    v = pd.DataFrame()
    v["i0"] = 0.099844*((((((14.27206993103027344)) - (((data["percent_atom_in"]) * ((10.0)))))) - (((3.0) * (data["SG_227"])))))
    v["i1"] = 0.100000*((((((((((((data["percent_atom_al"]) - (data["percent_atom_ga"]))) - (data["SG_194"]))) * 2.0)) * 2.0)) + (data["percent_atom_al"]))/2.0))
    v["i2"] = 0.097382*(((((data["SG_167"]) + (((np.cos((data["lattice_angle_beta_degree"]))) * 2.0)))) - ((((data["percent_atom_ga"]) + (data["SG_227"]))/2.0))))
    v["i3"] = 0.100000*((((((data["percent_atom_in"]) - (np.sin((((((data["percent_atom_in"]) * 2.0)) * 2.0)))))) + (data["a0"]))/2.0))
    v["i4"] = 0.094784*(((np.sin((((-3.0) * (((np.cos((data["lattice_angle_alpha_degree"]))) * 2.0)))))) * (data["lattice_angle_alpha_degree"])))
    v["i5"] = 0.100000*((((((((data["SG_194"]) - ((((data["SG_12"]) + (np.cos((1.0))))/2.0)))) + (data["SG_167"]))/2.0)) / 2.0))
    v["i6"] = 0.100000*((((((((((data["SG_206"]) + (data["SG_227"]))/2.0)) - (((data["lattice_angle_gamma_degree"]) * (data["SG_227"]))))) / 2.0)) / 2.0))
    v["i7"] = 0.100000*(((((((data["percent_atom_in"]) / 2.0)) + (((data["SG_33"]) - (((data["a1"]) * 2.0)))))) * (data["percent_atom_ga"])))
    v["i8"] = 0.098926*((((((((np.sin((data["SG_206"]))) + (np.sin((((3.0) + (data["SG_12"]))))))/2.0)) / 2.0)) / 2.0))
    v["i9"] = 0.089588*(((np.cos((((data["percent_atom_ga"]) - ((5.0)))))) * (np.sin((np.sin((np.sin((data["lattice_vector_3_ang"])))))))))
    v["i10"] = 0.100000*(((((np.sin((((((data["lattice_vector_2_ang"]) * 2.0)) * (data["SG_12"]))))) * (data["percent_atom_ga"]))) * 2.0))
    v["i11"] = 0.092596*(((((data["percent_atom_al"]) * (np.sin((((data["lattice_vector_1_ang"]) * ((-1.0*((data["SG_194"])))))))))) * (data["lattice_vector_1_ang"])))
    v["i12"] = 0.098887*((((((((data["percent_atom_in"]) * (data["percent_atom_in"]))) + (data["SG_194"]))/2.0)) * (((data["percent_atom_in"]) * (data["percent_atom_in"])))))
    v["i13"] = 0.100000*(((((((((data["percent_atom_al"]) / 2.0)) / 2.0)) - (data["percent_atom_in"]))) * (data["percent_atom_al"])))
    v["i14"] = 0.099355*(((data["SG_12"]) * ((((-2.0) + (((((0.15635852515697479)) + (data["number_of_total_atoms"]))/2.0)))/2.0))))
    v["i15"] = 0.100000*(((((((data["SG_194"]) * ((((0.04788995906710625)) * (np.cos((data["lattice_vector_3_ang"]))))))) * 2.0)) * 2.0))
    v["i16"] = 0.100000*(((((np.cos(((3.51841187477111816)))) - (np.cos((data["lattice_vector_3_ang"]))))) * (data["a1"])))
    v["i17"] = 0.058625*(((((((((data["SG_227"]) + (((data["SG_227"]) / 2.0)))) / 2.0)) / 2.0)) / 2.0))
    v["i18"] = 0.078531*(((np.sin((data["lattice_vector_3_ang"]))) * ((((data["percent_atom_al"]) + (np.sin((np.sin((np.cos(((4.28404331207275391)))))))))/2.0))))
    v["i19"] = 0.100000*(((np.cos((((data["a2"]) * 2.0)))) * (((((np.sin((((data["lattice_angle_alpha_degree"]) * 2.0)))) * 2.0)) * 2.0))))
    v["i20"] = 0.100000*(((((np.sin((-3.0))) * (np.sin((((data["number_of_total_atoms"]) * 2.0)))))) * (np.sin((data["a2"])))))
    v["i21"] = 0.077710*(((data["SG_194"]) * (((((((data["percent_atom_ga"]) * (data["percent_atom_ga"]))) - (data["percent_atom_al"]))) * (data["percent_atom_ga"])))))
    v["i22"] = 0.100000*(((((np.sin((((data["lattice_angle_alpha_degree"]) * 2.0)))) * (np.cos(((((2.67748546600341797)) * (data["a0"]))))))) * 2.0))
    v["i23"] = 0.095722*(((data["percent_atom_ga"]) * (((((data["percent_atom_al"]) * (data["percent_atom_ga"]))) * (data["percent_atom_al"])))))
    v["i24"] = 0.100000*(((data["percent_atom_ga"]) * (((np.cos(((9.58405303955078125)))) + (np.cos((((data["percent_atom_ga"]) - (data["a2"])))))))))
    v["i25"] = 0.088435*(((((data["a2"]) * (np.cos((data["lattice_angle_gamma_degree"]))))) * (((np.cos((data["lattice_vector_3_ang"]))) / 2.0))))
    v["i26"] = 0.084724*(((data["percent_atom_in"]) * (((data["percent_atom_in"]) * (np.sin((np.cos((((data["lattice_angle_alpha_degree"]) * ((7.0))))))))))))
    v["i27"] = 0.099980*((-1.0*((((((np.cos((data["SG_167"]))) * (np.cos((data["SG_206"]))))) * (np.cos(((4.69414234161376953)))))))))
    v["i28"] = 0.100000*(((0.0) * (0.0)))
    v["i29"] = 0.100000*(((0.0) * (0.0)))
    v["i30"] = 0.100000*(((0.0) * ((0.0))))
    v["i31"] = 0.100000*(((0.0) * (0.0)))
    v["i32"] = 0.011057*((((-1.0*((data["SG_227"])))) * ((((0.00905871577560902)) * ((((11.05083274841308594)) + ((0.00905871577560902))))))))
    v["i33"] = 0.100000*(((0.0) * (0.0)))
    v["i34"] = 0.100000*((((0.03779769688844681)) * (((((data["percent_atom_in"]) * (((data["percent_atom_in"]) - (data["SG_12"]))))) * (data["percent_atom_in"])))))
    v["i35"] = 0.000664*(((((np.sin(((((1.0) + (np.cos((3.0))))/2.0)))) * 2.0)) / 2.0))
    v["i36"] = 0.081539*(((data["a0"]) * (((((np.sin((data["a0"]))) * (data["a1"]))) * ((-1.0*((data["a1"]))))))))
    v["i37"] = 0.061848*(np.sin((np.sin((np.sin((np.sin((np.sin(((((-1.0*(((12.55811977386474609))))) * 2.0))))))))))))
    v["i38"] = 0.000000*(((data["SG_227"]) * (np.sin((((((np.sin((np.sin((-3.0))))) / 2.0)) / 2.0))))))
    v["i39"] = 0.100000*(((np.cos(((7.88480567932128906)))) * (np.sin((((((((data["lattice_vector_3_ang"]) * 2.0)) * 2.0)) * 2.0))))))
    v["i40"] = 0.100000*(((((data["SG_227"]) - (data["a2"]))) * ((((0.01647591963410378)) - (((data["SG_227"]) * (data["a2"])))))))
    v["i41"] = 0.100000*((((0.10676624625921249)) * ((((data["SG_227"]) + (((-3.0) * (((data["SG_227"]) * ((0.10676624625921249)))))))/2.0))))
    v["i42"] = 0.091424*(((((data["percent_atom_ga"]) * (((np.sin((((data["lattice_angle_gamma_degree"]) * 2.0)))) * (data["percent_atom_ga"]))))) * (data["a2"])))
    v["i43"] = 0.085310*(((((np.cos((data["lattice_angle_alpha_degree"]))) * ((4.0)))) * (np.cos(((((4.0)) * (data["lattice_vector_1_ang"])))))))
    v["i44"] = 0.100000*(((((data["a2"]) + (np.cos((data["lattice_angle_beta_degree"]))))) * (np.sin((np.sin((((data["lattice_angle_beta_degree"]) * 2.0))))))))
    v["i45"] = 0.071752*((((((0.00246048043482006)) * (data["lattice_angle_gamma_degree"]))) * (((((data["lattice_angle_gamma_degree"]) - (data["a2"]))) * ((4.0))))))
    v["i46"] = 0.086833*(((np.sin((((((data["lattice_angle_alpha_degree"]) * ((7.0)))) * 2.0)))) * (((np.cos((data["lattice_angle_beta_degree"]))) * 2.0))))
    v["i47"] = 0.051045*(((((data["a2"]) + (((((data["a1"]) * 2.0)) * (3.0))))) * (np.cos((data["lattice_angle_beta_degree"])))))
    v["i48"] = 0.100000*(((((data["a2"]) + ((3.70830512046813965)))) * (((data["a2"]) * (((-3.0) * ((0.00166296993847936))))))))
    v["i49"] = 0.091873*(np.sin((((data["a0"]) * ((-1.0*((data["SG_167"]))))))))
    v["i50"] = 0.100000*(((data["SG_206"]) * ((((data["SG_206"]) + (np.cos((((np.cos((data["percent_atom_ga"]))) + ((2.36559081077575684)))))))/2.0))))
    v["i51"] = 0.100000*(((((data["lattice_vector_1_ang"]) * ((0.00232100533321500)))) * ((-1.0*((np.sin(((((10.26993465423583984)) * (data["lattice_vector_1_ang"]))))))))))
    v["i52"] = 0.071362*(((np.cos(((14.16380882263183594)))) * (np.sin(((((((14.16378784179687500)) * (data["a2"]))) * (-2.0)))))))
    v["i53"] = 0.100000*(((((np.cos((data["lattice_angle_gamma_degree"]))) * (data["a0"]))) * (((data["a0"]) + (np.cos(((9.62785911560058594))))))))
    v["i54"] = 0.064055*(((((data["lattice_angle_alpha_degree"]) * (np.sin((((((data["lattice_angle_gamma_degree"]) * 2.0)) * ((6.0)))))))) * (data["lattice_angle_gamma_degree"])))
    v["i55"] = 0.100000*(((1.0) * (0.0)))
    v["i56"] = 0.100000*(((data["lattice_angle_gamma_degree"]) * (((((np.cos(((4.62204456329345703)))) * (data["a2"]))) * (data["SG_227"])))))
    v["i57"] = 0.069682*((((((14.92972660064697266)) * (data["lattice_vector_2_ang"]))) * (((data["SG_227"]) * ((((0.00005364419484977)) * ((14.92972946166992188))))))))
    v["i58"] = 0.099961*((((((0.00800371356308460)) * (np.sin((((np.sin((((data["lattice_vector_1_ang"]) / 2.0)))) * 2.0)))))) * 2.0))
    v["i59"] = 0.100000*(((((data["lattice_angle_gamma_degree"]) * (np.cos((((((((data["lattice_vector_3_ang"]) * 2.0)) * 2.0)) * 2.0)))))) * ((0.00879764743149281))))
    v["i60"] = 0.100000*(((data["lattice_angle_gamma_degree"]) * (((np.cos((((data["lattice_vector_3_ang"]) * 2.0)))) * (((data["lattice_angle_gamma_degree"]) * ((0.00563264032825828))))))))
    v["i61"] = 0.100000*(((0.0) * (0.0)))
    v["i62"] = 0.100000*(((0.0) * (0.0)))
    v["i63"] = 0.100000*(((0.0) * (0.0)))
    v["i64"] = 0.100000*(((0.0) * (0.0)))
    v["i65"] = 0.100000*(((np.cos(((((((((((3.13083600997924805)) * (data["lattice_vector_1_ang"]))) * 2.0)) * 2.0)) * 2.0)))) * ((0.02286315523087978))))
    v["i66"] = 0.100000*(((data["percent_atom_al"]) - (data["percent_atom_al"])))
    v["i67"] = 0.100000*(((0.0) - (0.0)))
    v["i68"] = 0.044696*((((((((10.98523330688476562)) * (((np.sin(((10.98523616790771484)))) - (-1.0))))) * 2.0)) * 2.0))
    v["i69"] = 0.100000*((((0.0)) * (data["a2"])))
    v["i70"] = 0.097265*(((((np.cos(((4.70197057723999023)))) * ((10.0)))) * (np.sin(((((11.15155506134033203)) * (data["percent_atom_al"])))))))
    v["i71"] = 0.100000*(((data["percent_atom_in"]) * (((data["a0"]) * ((((-1.0*((np.sin(((7.0))))))) + (data["percent_atom_in"])))))))
    v["i72"] = 0.099277*(((((((((data["percent_atom_al"]) * (data["SG_227"]))) - (data["SG_194"]))) * (data["percent_atom_al"]))) * (data["percent_atom_in"])))
    v["i73"] = 0.085192*(((((((np.sin((data["lattice_vector_1_ang"]))) + (np.cos((data["a2"]))))) * ((0.03267288953065872)))) * (data["lattice_angle_gamma_degree"])))
    v["i74"] = 0.090018*((((((data["a0"]) + (np.sin((((((data["a0"]) * 2.0)) * 2.0)))))/2.0)) * (np.sin(((12.51333713531494141))))))
    v["i75"] = 0.100000*(((((data["a1"]) * (((data["a1"]) * (((data["a1"]) - (np.sin((data["a1"]))))))))) / 2.0))
    v["i76"] = 0.048193*(((((np.sin((data["lattice_angle_beta_degree"]))) + (-1.0))) * ((((np.sin((-1.0))) + (-2.0))/2.0))))
    v["i77"] = 0.062668*(((((np.sin(((((10.99305820465087891)) * (np.sin((data["percent_atom_in"]))))))) * 2.0)) * (np.cos(((7.88768100738525391))))))
    v["i78"] = 0.100000*((((0.0) + (0.0))/2.0))
    v["i79"] = 0.100000*(((((((np.sin((np.sin((data["percent_atom_al"]))))) - (np.cos((1.0))))) * (data["percent_atom_al"]))) / 2.0))
    v["i80"] = 0.100000*(((((((((((((((((data["lattice_angle_gamma_degree"]) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0)) / 2.0))
    v["i81"] = 0.098164*(((((data["percent_atom_in"]) * (data["percent_atom_ga"]))) * (((np.cos((((data["lattice_vector_3_ang"]) * 2.0)))) * (data["percent_atom_in"])))))
    v["i82"] = 0.100000*(((0.0) * (0.0)))
    v["i83"] = 0.100000*(((((np.sin((data["lattice_angle_beta_degree"]))) - (((data["lattice_angle_beta_degree"]) - (np.sin((data["SG_12"]))))))) * (data["SG_12"])))
    v["i84"] = 0.100000*(((((((((data["percent_atom_ga"]) * (((data["percent_atom_ga"]) * (data["SG_167"]))))) / 2.0)) * (data["percent_atom_ga"]))) / 2.0))
    v["i85"] = 0.100000*(((((np.sin((((data["lattice_vector_3_ang"]) * ((7.18604373931884766)))))) * 2.0)) * (((np.cos((data["lattice_angle_alpha_degree"]))) * 2.0))))
    v["i86"] = 0.100000*((((0.01680851355195045)) * (np.sin((((((((data["lattice_vector_3_ang"]) * 2.0)) * (data["lattice_vector_1_ang"]))) * (data["lattice_vector_1_ang"])))))))
    v["i87"] = 0.099980*(((np.sin((((data["number_of_total_atoms"]) * 2.0)))) * (((np.sin((np.sin(((((7.85724353790283203)) * 2.0)))))) * 2.0))))
    v["i88"] = 0.067103*(((np.cos(((((12.10680103302001953)) * ((14.13893222808837891)))))) * (np.cos(((((12.10680103302001953)) * (data["a2"])))))))
    v["i89"] = 0.052881*((((((data["lattice_angle_gamma_degree"]) + (data["SG_227"]))/2.0)) * ((0.02614975534379482))))
    v["i90"] = 0.100000*(((0.0) * (0.0)))
    v["i91"] = 0.002715*((((((((-1.0*(((0.00171303795650601))))) * 2.0)) * 2.0)) - ((0.00171303795650601))))
    v["i92"] = 0.072866*(((((((((((((1.0) - ((0.99883699417114258)))) * (data["SG_206"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0))
    v["i93"] = 0.100000*((((14.73002338409423828)) * (((np.sin(((3.14319562911987305)))) * (np.cos(((((13.76105785369873047)) * (data["lattice_vector_1_ang"])))))))))
    v["i94"] = 0.100000*(((0.0) * (0.0)))
    v["i95"] = 0.100000*((((((2.95604181289672852)) * (np.cos((data["lattice_angle_alpha_degree"]))))) * (np.cos((((data["lattice_vector_1_ang"]) * (data["lattice_vector_1_ang"])))))))
    v["i96"] = 0.100000*(((0.0) * (0.0)))
    v["i97"] = 0.100000*(((((data["SG_227"]) - (data["a2"]))) * (((data["SG_227"]) * ((((0.20469550788402557)) - (data["a2"])))))))
    v["i98"] = 0.073530*(((((((np.cos((data["a0"]))) - (data["a0"]))) - (data["a0"]))) * (np.cos((data["lattice_angle_alpha_degree"])))))
    v["i99"] = 0.033952*(((((data["SG_227"]) * ((0.02502322755753994)))) * (((((data["SG_227"]) - ((3.0)))) - (data["a0"])))))
    return v.sum(axis=1).values

In [ ]:
print((mean_squared_error(target_fe,FE1(train))))
print((mean_squared_error(target_be,BE1(train))))
print((mean_squared_error(target_fe,FE2(train))))
print((mean_squared_error(target_be,BE2(train))))

In [ ]:
fe = (np.minimum(0.505129431221,np.maximum(0,FE1(test)))+
      np.minimum(0.505129431221,np.maximum(0,FE2(test))))/2
be = (np.minimum(1.83834084662,np.maximum(0,BE1(test)))+
      np.minimum(1.83834084662,np.maximum(0,BE2(test))))/2
sub = pd.read_csv('../input/sample_submission.csv')
sub['formation_energy_ev_natom'] = np.expm1(fe)
sub['bandgap_energy_ev'] = np.expm1(be)
sub.to_csv('gpsubmission.csv', index=False)